- Идентификатор заявки: SkillFactory_Id
- Дата рождения: BirthDate
- Адрес проживания: -
- Исключен из расчета
- Образование: education
- Тип занятости: employment status
- Стаж работы: Value
- Дата начала работы: JobStartDate
- Должность: Position
- Рабочий телефон: -
- Исключен из расчета
- Ежемесячный доход: MonthProfit
- Ежемесячный расход: MonthExpense
- Пол: Gender
    - Значения: 0 = М, 1 = Ж
- Семейное положение: Family status
- Кол-во детей младше 18 лет: ChildCount
- СНИЛС: SNILS
    - Значения: 0 = не указан в анкете, 1 = указан
- Параметры заказа
    - Сумма заказа: Loan_amount
    - Срок кредита: Loan_term
    - Категория товара: Goods_category
    - Код магазина: Merch_code
- Решение банка А: BankA_decision
    - Значения: denied = отказ банка по заявке, success = одобрение банком заявки, error = банк вернул ошику
- Решение банка B: BankB_decision
    - Значения: denied = отказ банка по заявке, success = одобрение банком заявки, error = банк вернул ошибку
- Решение банка C: BankC_decision
    - Значения: denied = отказ банка по заявке, success = одобрение банком заявки, error = банк вернул ошибку
- Решение банка D: BankD_decision
    - Значения: denied = отказ банка по заявке, success = одобрение банком заявки, error = банк вернул ошибку
- Решение банка E: BankE_decision
    - Значения: denied = отказ банка по заявке, success = одобрение банком заявки, error = банк вернул ошибку



In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [85]:
def drop_outliers(df, column_name):
    """
    Drop outliers from a DataFrame based on a specified column.

    Parameters:
    - df: DataFrame
    - column_name: str, the column to check for outliers

    Returns:
    - DataFrame without outliers
    """

    # Calculate the 95% confidence interval
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # Identify and drop outliers
    outliers = (df[column_name] < lower_bound) | (df[column_name] > upper_bound)
    df_no_outliers = df[~outliers]

    return df_no_outliers

In [86]:
df = pd.read_csv('../data/datasets/SF_Mandarin_dataset_ver3.csv', sep=';', index_col=0)


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 1 to 8787
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   birth_date         6464 non-null   datetime64[ns]
 1   education          6464 non-null   object        
 2   employment_status  6464 non-null   object        
 3   value              6464 non-null   object        
 4   job_start_date     6464 non-null   datetime64[ns]
 5   position           6464 non-null   object        
 6   month_profit       6464 non-null   float64       
 7   month_expense      6464 non-null   float64       
 8   gender             6464 non-null   float64       
 9   family_status      6464 non-null   object        
 10  child_count        6464 non-null   float64       
 11  snils              6464 non-null   float64       
 12  bank_a_decision    6464 non-null   object        
 13  bank_b_decision    6464 non-null   object        
 14  bank_c_decisi

In [88]:
df.dropna(inplace=True)


In [89]:
df.drop_duplicates(inplace=True)

In [90]:
df.index = df.index.astype(int)

In [91]:
import re

def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

In [92]:
def convert_from_camel_case_to_snake_case(df):
    """
    Convert from camel case to snake case
    """
    df.columns = df.columns.to_series().apply(camel_to_snake)
    df.columns = df.columns.str.replace(' ', '_')

In [93]:
convert_from_camel_case_to_snake_case(df)

In [94]:
df['birth_date'] = pd.to_datetime(df['birth_date'])
df['job_start_date'] = pd.to_datetime(df['job_start_date'])

In [76]:
def columns_to_type(df, columns: list[str], dtype: str):
    """
    Set columns to type
    """
    for column in columns:
        df[column] = df[column].astype(dtype)

In [77]:
# to int64
columns_to_type(df, ['month_profit', 'month_expense', 'loan_amount'], 'UInt64')

In [78]:
columns_to_type(df, ['snils', 'gender', 'merch_code', 'child_count', 'loan_term'], 'UInt8')
columns_to_type(df, ['family_status', 'goods_category', 'position', 'employment_status', 'education', 'snils', 'gender'], 'category')

In [79]:
for i in ['a', 'b', 'c', 'd', 'e']:
    df[f'bank_{i}_decision'] = df[f'bank_{i}_decision'].astype('category')

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 1 to 8787
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   birth_date         6464 non-null   datetime64[ns]
 1   education          6464 non-null   category      
 2   employment_status  6464 non-null   category      
 3   value              6464 non-null   object        
 4   job_start_date     6464 non-null   datetime64[ns]
 5   position           6464 non-null   category      
 6   month_profit       6464 non-null   UInt64        
 7   month_expense      6464 non-null   UInt64        
 8   gender             6464 non-null   category      
 9   family_status      6464 non-null   category      
 10  child_count        6464 non-null   UInt8         
 11  snils              6464 non-null   category      
 12  bank_a_decision    6464 non-null   category      
 13  bank_b_decision    6464 non-null   category      
 14  bank_c_decisi

In [81]:
df.to_parquet('../data/datasets/preprocessed_dataset.parquet', compression='gzip',)

In [82]:
df = pd.read_parquet('../data/datasets/preprocessed_dataset.parquet')

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 1 to 8787
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   birth_date         6464 non-null   datetime64[ns]
 1   education          6464 non-null   category      
 2   employment_status  6464 non-null   category      
 3   value              6464 non-null   object        
 4   job_start_date     6464 non-null   datetime64[ns]
 5   position           6464 non-null   category      
 6   month_profit       6464 non-null   UInt64        
 7   month_expense      6464 non-null   UInt64        
 8   gender             6464 non-null   uint8         
 9   family_status      6464 non-null   category      
 10  child_count        6464 non-null   UInt8         
 11  snils              6464 non-null   uint8         
 12  bank_a_decision    6464 non-null   category      
 13  bank_b_decision    6464 non-null   category      
 14  bank_c_decisi

In [ ]:
df['Position'] = df['Position'].str.lower()


In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
summary_stats = df.describe()
print(summary_stats)

In [ ]:
# Plot histograms for numeric variables
numeric_vars = ['MonthProfit', 'MonthExpense', 'ChildCount', 'SNILS', 'Loan_amount', 'Loan_term', 'Merch_code']
df[numeric_vars].hist(bins=20, figsize=(15, 10))
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(df['MonthProfit'],)
plt.title(f'Distribution of MonthProfit')
plt.show()

In [ ]:
df[df['MonthProfit'] > 1000000]

In [ ]:
# Explore categorical variables
cat_vars = ['education', 'employment status', 'Family status', 'Goods_category', 'SNILS', 'Gender']
for var in cat_vars:
    plt.figure(figsize=(20, 6))
    sns.countplot(x=var, data=df)
    plt.title(f'Distribution of {var}')
    plt.show()

In [ ]:
# Explore bank decisions
bank_decision = ['BankA_decision', 'BankB_decision', 'BankC_decision', 'BankD_decision', 'BankE_decision']
for bank in bank_decision:
    plt.figure(figsize=(10, 6))
    sns.countplot(x=bank, data=df)
    plt.title(f'Distribution of {bank}')
    plt.show()

In [ ]:
# List of bank decision columns
bank_decision_columns = ['BankA_decision', 'BankB_decision', 'BankC_decision', 'BankD_decision', 'BankE_decision']

# Create a new column 'Overall_decision' with the overall decision for each row
df['Overall_decision'] = df[bank_decision_columns].apply(lambda row: 'success' if 'success' in row.values else 'denied', axis=1)
df['Overall_decision'] = df['Overall_decision'].astype('category')

# Count the overall decisions
overall_decision_count = df['Overall_decision'].value_counts()

# Display the results
print("Overall Bank Decisions:")
print(overall_decision_count)

In [ ]:
# Set the style for seaborn
sns.set(style="whitegrid")

# Create a bar plot comparing 'Family status' with 'ChildCount'
plt.figure(figsize=(20, 6))
sns.barplot(x='Family status', y='ChildCount', data=df, palette='viridis', hue='Goods_category')
plt.title('Comparison of Family Status and Child Count')
plt.xlabel('Family Status')
plt.ylabel('Child Count')
plt.show()

In [ ]:
# Set the style for seaborn
sns.set(style="whitegrid")

# Create a bar plot comparing 'MonthProfit' and 'MonthExpense'
plt.figure(figsize=(20, 6))
sns.barplot(x='Family status', y='MonthProfit', data=df, color='skyblue', label='MonthProfit', errorbar=None)
sns.barplot(x='Family status', y='MonthExpense', data=df, color='orange', label='MonthExpense', errorbar=None)
plt.title('Comparison of MonthProfit and MonthExpense by Family Status')
plt.xlabel('Family Status')
plt.ylabel('Amount')
plt.legend()
plt.show()

In [ ]:
# Select columns related to bank decisions
bank_decisions_columns = ['BankA_decision', 'BankB_decision', 'BankC_decision', 'BankD_decision', 'BankE_decision']

# Create a DataFrame containing only bank decision columns
bank_decisions_df = df[bank_decisions_columns]

# Map 'success' and 'denied' to numeric values (1 and 0)
bank_decisions_df_numeric = bank_decisions_df.map(lambda x: 1 if x == 'success' else 0)

# Calculate the correlation matrix
correlation_matrix = bank_decisions_df_numeric.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Heatmap for Bank Decisions')
plt.show()

In [ ]:
df['MonthExpense'].describe()

In [ ]:
df['Income_Expense_Ratio'] = np.where(df['MonthExpense'] != 0, df['MonthProfit'] / df['MonthExpense'], np.nan)

# Display the updated DataFrame
print(df[['MonthProfit', 'MonthExpense', 'Income_Expense_Ratio']])

In [ ]:
df['Income_Expense_Ratio'].isna().sum()

In [ ]:
# Set the style for seaborn
sns.set(style="whitegrid")

# Filter the DataFrame for ratios less than or equal to 10
filtered_df = df[df['Income_Expense_Ratio'] <= 10]

# Create a boxplot for 'Income_Expense_Ratio'
plt.figure(figsize=(10, 6))
sns.boxplot(x=filtered_df['Income_Expense_Ratio'], color='skyblue')
plt.title('Boxplot of Income-to-Expense Ratio (<= 10)')
plt.xlabel('Income-to-Expense Ratio')
plt.show()

In [ ]:
print(df.info())

In [ ]:
# Create a scatter plot for 'Loan_amount' and 'Loan_term'
plt.figure(figsize=(10, 6))
plt.scatter(df['Loan_amount'], df['Loan_term'], color='skyblue', alpha=0.5, )
plt.title('Scatter Plot of Loan Amount vs. Loan Term')
plt.xlabel('Loan Amount')
plt.ylabel('Loan Term')
plt.show()

In [ ]:
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# List of bank decision columns
bank_decision_columns = ['BankA_decision', 'BankB_decision', 'BankC_decision', 'BankD_decision', 'BankE_decision']

# Calculate point-biserial correlation with loan term for each bank decision
label_encoder = LabelEncoder()
for bank in bank_decision_columns:
    df[bank] = label_encoder.fit_transform(df[bank])

    # Calculate point-biserial correlation with loan term for 'BankA_decision_encoded'
    correlation, _ = stats.pointbiserialr(df[bank], df['Loan_term'])

    # Display the correlation coefficient
    print(f"Correlation between BankA_decision and Loan_term: {correlation}")

    # Visualize the relationship using a box plot
    plt.figure(figsize=(12, 8))
    sns.boxplot(x='Loan_term', y=bank, data=df, orient='h')
    plt.title(f'Box Plot of Loan Term by {bank}')
    plt.xlabel('Loan Term')
    plt.ylabel(f'{bank} Decision (Encoded)')
    plt.show()
        


In [ ]:
df.to_csv('datasets/SF_Mandarin_dataset_processed.csv', sep=',')